In [34]:
import pandas as pd

columns = ['query', 'seed_ortholog', 'evalue', 'score', 'predicted_name', 'go_terms', 'kegg_ko', 'bigg_reactions',
           'tax_scope', 'eggnog_ogs', 'best_og', 'cog_cat', 'eggnog_hmmm_desc']

filename = '/Users/olgabot/Downloads/tick0.fake_ref_transcriptome.fasta.emapper.annotations.tsv'
tick0_annotations = pd.read_table(filename, header=None, names=columns)
print(tick0_annotations.shape)
tick0_annotations.head()

(3861, 13)


,query,seed_ortholog,evalue,score,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope,eggnog_ogs,best_og,cog_cat,eggnog_hmmm_desc
0,PB.3.1|1001_0|path0:1-4172(+)|HQ_sampleytBMi1Q...,6945.ISCW018217-PA,1.400000e-66,260.8,SDC,"GO:0000902,GO:0000904,GO:0001667,GO:0003008,GO...",K16336,NaN,artNOG[6],"0386E@artNOG,0A1US@biNOG,0IW5I@euNOG,0VCQE@meN...",NA|NA|NA,T,Cell surface proteoglycan (By similarity)
1,PB.8.1|1008_0|path3:1-3937(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,3.500000e-149,535.0,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel
2,PB.9.1|1008_0|path4:1-3923(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,5.900000e-149,534.3,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel
3,PB.11.2|101_0|path4:1-2908(+)|HQ_sampleytBMi1Q...,6945.ISCW018610-PA,3.300000e-38,165.2,NaN,NaN,NaN,NaN,artNOG[6],"0A3DQ@biNOG,0VF6Y@meNOG,11U5P@NOG,13M9C@opiNOG...",NA|NA|NA,Y,nucleolar and coiled-body phosphoprotein 1
4,PB.14.1|1011_0|path0:183-1775(-)|HQ_sampleytBM...,7227.FBpp0100183,3.100000e-56,224.9,NaN,"GO:0003674,GO:0003824,GO:0003954,GO:0005575,GO...",K03881,NaN,artNOG[6],"03GA4@artNOG,0AYRX@biNOG,0H2PW@dipNOG,0KFRK@eu...",NA|NA|NA,C,"NADH-Ubiquinone/plastoquinone (complex I), var..."


In [35]:
tick0_annotations.sort_values('score', ascending=False)

,query,seed_ortholog,evalue,score,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope,eggnog_ogs,best_og,cog_cat,eggnog_hmmm_desc
94,PB.247.1|1214_0|path0:1-6018(+)|HQ_sampleytBMi...,6945.ISCW001340-PA,0.000000e+00,2252.2,MHC1,NaN,K17751,NaN,artNOG[6],"032CV@artNOG,09T3R@biNOG,0V42H@meNOG,12P08@opi...",NA|NA|NA,Z,Myosin heavy chain
81,PB.222.2|1192_1|path0:1-4445(+)|HQ_sampleytBMi...,6945.ISCW009225-PA,0.000000e+00,1801.2,NaN,"GO:0003008,GO:0003674,GO:0003824,GO:0005215,GO...",K05853,NaN,artNOG[6],"03265@artNOG,09VH3@biNOG,0V6GC@meNOG,12N40@opi...",NA|NA|NA,P,Calcium-transporting ATPase sarcoplasmic endop...
80,PB.222.1|1192_1|path0:1-5152(+)|HQ_sampleytBMi...,6945.ISCW009225-PA,0.000000e+00,1763.4,NaN,"GO:0003008,GO:0003674,GO:0003824,GO:0005215,GO...",K05853,NaN,artNOG[6],"03265@artNOG,09VH3@biNOG,0V6GC@meNOG,12N40@opi...",NA|NA|NA,P,Calcium-transporting ATPase sarcoplasmic endop...
1749,PB.3990.1|83_0|path1:1-3297(+)|HQ_sampleytBMi1...,34740.HMEL017480-PA,0.000000e+00,1726.8,SXC,"GO:0003674,GO:0003824,GO:0005975,GO:0006464,GO...",K09667,NaN,artNOG[6],"034XH@artNOG,09UN5@biNOG,0SRIQ@inNOG,0TA41@lep...",NA|NA|NA,"G, O, T",Tetratricopeptide repeat
742,PB.1632.1|2433_1|path0:1-2918(+)|HQ_sampleytBM...,6945.ISCW000137-PA,0.000000e+00,1704.9,MYO22,"GO:0003674,GO:0003824,GO:0005575,GO:0005622,GO...",NaN,NaN,artNOG[6],"033YK@artNOG,09YTB@biNOG,0V61Q@meNOG,12P2Y@opi...",NA|NA|NA,Z,MYSc
3374,PB.11790.1|HQ_sampleytBMi1Qu|cb968_c0/f7p2/299...,6945.ISCW022112-PA,0.000000e+00,1703.3,COPB2,"GO:0000139,GO:0005575,GO:0005622,GO:0005623,GO...",K17302,NaN,artNOG[6],"035FM@artNOG,09WCW@biNOG,0V7BC@meNOG,0XNNY@NOG...",NA|NA|NA,U,Coatomer protein complex subunit beta 2
1447,PB.3522.2|562_0|path38:43-5162(+)|HQ_sampleytB...,7176.CPIJ003589-PA,0.000000e+00,1608.6,NaN,"GO:0005575,GO:0005622,GO:0005623,GO:0005856,GO...",K08770,NaN,artNOG[6],"032W7@artNOG,09XHH@biNOG,0SR8C@inNOG,0V8FJ@meN...",NA|NA|NA,O,K08770 ubiquitin C
460,PB.934.1|180_0|path1:1-2704(+)|HQ_sampleytBMi1...,6945.ISCW023297-PA,0.000000e+00,1602.0,NaN,"GO:0000375,GO:0000377,GO:0000381,GO:0000398,GO...",K12867,NaN,artNOG[6],"035D8@artNOG,09YVS@biNOG,0V9TJ@meNOG,0XPD9@NOG...",NA|NA|NA,A,HAT
1826,PB.4161.1|98_0|path0:1-2912(+)|HQ_sampleytBMi1...,6945.ISCW000885-PA,0.000000e+00,1458.0,GLYP,"GO:0000272,GO:0003674,GO:0003824,GO:0004645,GO...",K00688,NaN,artNOG[6],"0359U@artNOG,09TCR@biNOG,0V4BV@meNOG,12N63@opi...",NA|NA|NA,G,Phosphorylase is an important allosteric enzym...
2712,PB.8901.1|HQ_sampleytBMi1Qu|cb338_c0/f3p0/3497...,7739.JGI223894,0.000000e+00,1437.6,NaN,NaN,NaN,NaN,artNOG[6],"09UYJ@biNOG,0DG82@chorNOG,0V5YF@meNOG,12NJH@op...",NA|NA|NA,K,"polymerase (RNA) I polypeptide B, 128kDa"


### Read human-based taxon annotation

In [36]:
import pandas as pd

columns = ['query', 'seed_ortholog', 'evalue', 'score', 'predicted_name', 'go_terms', 'kegg_ko', 'bigg_reactions',
           'tax_scope', 'eggnog_ogs', 'best_og', 'cog_cat', 'eggnog_hmmm_desc']

filename = '/Users/olgabot/Downloads/tick0.fake_ref_transcriptome.fasta.emapper.annotations'
tax_human = pd.read_table(filename, header=None, names=columns)
print(tax_human.shape)
tax_human.head()

(3861, 13)


,query,seed_ortholog,evalue,score,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope,eggnog_ogs,best_og,cog_cat,eggnog_hmmm_desc
0,PB.3.1|1001_0|path0:1-4172(+)|HQ_sampleytBMi1Q...,6945.ISCW018217-PA,1.400000e-66,260.8,SDC,"GO:0000902,GO:0000904,GO:0001667,GO:0003008,GO...",K16336,NaN,artNOG[6],"0386E@artNOG,0A1US@biNOG,0IW5I@euNOG,0VCQE@meN...",NA|NA|NA,T,Cell surface proteoglycan (By similarity)
1,PB.8.1|1008_0|path3:1-3937(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,3.500000e-149,535.0,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel
2,PB.9.1|1008_0|path4:1-3923(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,5.900000e-149,534.3,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel
3,PB.11.2|101_0|path4:1-2908(+)|HQ_sampleytBMi1Q...,6945.ISCW018610-PA,3.300000e-38,165.2,NOLC1,"GO:0000278,GO:0000280,GO:0001071,GO:0002119,GO...",K11294,NaN,meNOG[21],"0A3DQ@biNOG,0VF6Y@meNOG,11U5P@NOG,13M9C@opiNOG...",NA|NA|NA,Y,nucleolar and coiled-body phosphoprotein 1
4,PB.14.1|1011_0|path0:183-1775(-)|HQ_sampleytBM...,7227.FBpp0100183,3.100000e-56,224.9,NaN,"GO:0003674,GO:0003824,GO:0003954,GO:0005575,GO...",K03881,NaN,artNOG[6],"03GA4@artNOG,0AYRX@biNOG,0H2PW@dipNOG,0KFRK@eu...",NA|NA|NA,C,"NADH-Ubiquinone/plastoquinone (complex I), var..."


In [37]:
tax_human.sort_values('score', ascending=False)

,query,seed_ortholog,evalue,score,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope,eggnog_ogs,best_og,cog_cat,eggnog_hmmm_desc
94,PB.247.1|1214_0|path0:1-6018(+)|HQ_sampleytBMi...,6945.ISCW001340-PA,0.000000e+00,2252.2,MHC1,NaN,K17751,NaN,artNOG[6],"032CV@artNOG,09T3R@biNOG,0V42H@meNOG,12P08@opi...",NA|NA|NA,Z,Myosin heavy chain
81,PB.222.2|1192_1|path0:1-4445(+)|HQ_sampleytBMi...,6945.ISCW009225-PA,0.000000e+00,1801.2,NaN,"GO:0003008,GO:0003674,GO:0003824,GO:0005215,GO...",K05853,NaN,artNOG[6],"03265@artNOG,09VH3@biNOG,0V6GC@meNOG,12N40@opi...",NA|NA|NA,P,Calcium-transporting ATPase sarcoplasmic endop...
80,PB.222.1|1192_1|path0:1-5152(+)|HQ_sampleytBMi...,6945.ISCW009225-PA,0.000000e+00,1763.4,NaN,"GO:0003008,GO:0003674,GO:0003824,GO:0005215,GO...",K05853,NaN,artNOG[6],"03265@artNOG,09VH3@biNOG,0V6GC@meNOG,12N40@opi...",NA|NA|NA,P,Calcium-transporting ATPase sarcoplasmic endop...
1749,PB.3990.1|83_0|path1:1-3297(+)|HQ_sampleytBMi1...,34740.HMEL017480-PA,0.000000e+00,1726.8,SXC,"GO:0003674,GO:0003824,GO:0005975,GO:0006464,GO...",K09667,NaN,artNOG[6],"034XH@artNOG,09UN5@biNOG,0SRIQ@inNOG,0TA41@lep...",NA|NA|NA,"G, O, T",Tetratricopeptide repeat
742,PB.1632.1|2433_1|path0:1-2918(+)|HQ_sampleytBM...,6945.ISCW000137-PA,0.000000e+00,1704.9,MYO22,"GO:0003674,GO:0003824,GO:0005575,GO:0005622,GO...",NaN,NaN,artNOG[6],"033YK@artNOG,09YTB@biNOG,0V61Q@meNOG,12P2Y@opi...",NA|NA|NA,Z,MYSc
3374,PB.11790.1|HQ_sampleytBMi1Qu|cb968_c0/f7p2/299...,6945.ISCW022112-PA,0.000000e+00,1703.3,COPB2,"GO:0000139,GO:0005575,GO:0005622,GO:0005623,GO...",K17302,NaN,artNOG[6],"035FM@artNOG,09WCW@biNOG,0V7BC@meNOG,0XNNY@NOG...",NA|NA|NA,U,Coatomer protein complex subunit beta 2
1447,PB.3522.2|562_0|path38:43-5162(+)|HQ_sampleytB...,7176.CPIJ003589-PA,0.000000e+00,1608.6,NaN,"GO:0005575,GO:0005622,GO:0005623,GO:0005856,GO...",K08770,NaN,artNOG[6],"032W7@artNOG,09XHH@biNOG,0SR8C@inNOG,0V8FJ@meN...",NA|NA|NA,O,K08770 ubiquitin C
460,PB.934.1|180_0|path1:1-2704(+)|HQ_sampleytBMi1...,6945.ISCW023297-PA,0.000000e+00,1602.0,NaN,"GO:0000375,GO:0000377,GO:0000381,GO:0000398,GO...",K12867,NaN,artNOG[6],"035D8@artNOG,09YVS@biNOG,0V9TJ@meNOG,0XPD9@NOG...",NA|NA|NA,A,HAT
1826,PB.4161.1|98_0|path0:1-2912(+)|HQ_sampleytBMi1...,6945.ISCW000885-PA,0.000000e+00,1458.0,GLYP,"GO:0000272,GO:0003674,GO:0003824,GO:0004645,GO...",K00688,NaN,artNOG[6],"0359U@artNOG,09TCR@biNOG,0V4BV@meNOG,12N63@opi...",NA|NA|NA,G,Phosphorylase is an important allosteric enzym...
2712,PB.8901.1|HQ_sampleytBMi1Qu|cb338_c0/f3p0/3497...,7739.JGI223894,0.000000e+00,1437.6,POLR1B,"GO:0000003,GO:0000428,GO:0002119,GO:0002164,GO...",K03002,NaN,meNOG[21],"09UYJ@biNOG,0DG82@chorNOG,0V5YF@meNOG,12NJH@op...",NA|NA|NA,K,"polymerase (RNA) I polypeptide B, 128kDa"


In [38]:
mask = tick0_annotations != tax_human

tax_human[mask].dropna(how='all', axis=1).dropna(how='all', axis=0)

,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope
3,NOLC1,"GO:0000278,GO:0000280,GO:0001071,GO:0002119,GO...",K11294,NaN,meNOG[21]
7,CYTB,"GO:0000003,GO:0001666,GO:0003674,GO:0003824,GO...",K00412,NaN,meNOG[21]
8,CYTB,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",K00412,NaN,fiNOG[1]
9,CYTB,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",K00412,NaN,aveNOG[1]
13,NaN,NaN,"K13289,K16296,K16297",NaN,euNOG[57]
14,CBP1,"GO:0005575,GO:0005576,GO:0005622,GO:0005623,GO...",K16296,NaN,euNOG[57]
18,NaN,NaN,"K01049,K01050",NaN,euNOG[57]
24,NaN,NaN,NaN,NaN,euNOG[57]
25,DDX52,"GO:0003674,GO:0003676,GO:0003723,GO:0003724,GO...",K14779,NaN,meNOG[21]
28,KDELR3,"GO:0001932,GO:0001934,GO:0003674,GO:0005046,GO...",K10949,NaN,meNOG[21]


In [39]:
tick0_annotations[mask].dropna(how='all', axis=1).dropna(how='all', axis=0)

,tax_scope
3,artNOG[6]
7,artNOG[6]
8,artNOG[6]
9,artNOG[6]
13,artNOG[6]
14,artNOG[6]
18,artNOG[6]
24,artNOG[6]
25,artNOG[6]
28,artNOG[6]


In [40]:
tax_human.head()

,query,seed_ortholog,evalue,score,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope,eggnog_ogs,best_og,cog_cat,eggnog_hmmm_desc
0,PB.3.1|1001_0|path0:1-4172(+)|HQ_sampleytBMi1Q...,6945.ISCW018217-PA,1.400000e-66,260.8,SDC,"GO:0000902,GO:0000904,GO:0001667,GO:0003008,GO...",K16336,NaN,artNOG[6],"0386E@artNOG,0A1US@biNOG,0IW5I@euNOG,0VCQE@meN...",NA|NA|NA,T,Cell surface proteoglycan (By similarity)
1,PB.8.1|1008_0|path3:1-3937(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,3.500000e-149,535.0,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel
2,PB.9.1|1008_0|path4:1-3923(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,5.900000e-149,534.3,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel
3,PB.11.2|101_0|path4:1-2908(+)|HQ_sampleytBMi1Q...,6945.ISCW018610-PA,3.300000e-38,165.2,NOLC1,"GO:0000278,GO:0000280,GO:0001071,GO:0002119,GO...",K11294,NaN,meNOG[21],"0A3DQ@biNOG,0VF6Y@meNOG,11U5P@NOG,13M9C@opiNOG...",NA|NA|NA,Y,nucleolar and coiled-body phosphoprotein 1
4,PB.14.1|1011_0|path0:183-1775(-)|HQ_sampleytBM...,7227.FBpp0100183,3.100000e-56,224.9,NaN,"GO:0003674,GO:0003824,GO:0003954,GO:0005575,GO...",K03881,NaN,artNOG[6],"03GA4@artNOG,0AYRX@biNOG,0H2PW@dipNOG,0KFRK@eu...",NA|NA|NA,C,"NADH-Ubiquinone/plastoquinone (complex I), var..."


In [59]:
hash("asdf")

8877214956138409021

In [60]:
hash(1)

1

In [62]:
hash("TGTGAT")

3548232046661271295

In [55]:
from io import StringIO


s = '''A	RNA processing and modification
B	Chromatin Structure and dynamics
C	Energy production and conversion
D	Cell cycle control and mitosis
E	Amino Acid metabolism and transport
F	Nucleotide metabolism and transport
G	Carbohydrate metabolism and transport
H	Coenzyme metabolism
I	Lipid metabolism
J	Translation
K	Transcription
L	Replication and repair
M	Cell wall/membrane/envelope biogenesis
N	Cell motility
O	Post-translational modification, protein turnover, chaperone functions
P	Inorganic ion transport and metabolism
Q	Secondary Structure
T	Signal Transduction
U	Intracellular trafficking and secretion
V	Defense mechanisms
Y	Nuclear structure
W	Extracellular structures
Z	Cytoskeleton
R	General Functional Prediction only
S	Function Unknown'''

cog_categories = pd.read_table(StringIO(s), names=['letter', 'function'], index_col=0, squeeze=True)
cog_categories.head()

letter
A        RNA processing and modification
B       Chromatin Structure and dynamics
C       Energy production and conversion
D         Cell cycle control and mitosis
E    Amino Acid metabolism and transport
Name: function, dtype: object

In [57]:
tax_human['cog_category_full'] = tax_human.cog_cat.map(
    lambda x: '; '.join([cog_categories[i] for i in x.split(', ')]) if pd.notnull(x) else x)
tax_human['cog_category_full']

0                                     Signal Transduction
1                 Intracellular trafficking and secretion
2                 Intracellular trafficking and secretion
3                                       Nuclear structure
4                        Energy production and conversion
5                        Energy production and conversion
6                        Energy production and conversion
7                        Energy production and conversion
8                        Energy production and conversion
9                        Energy production and conversion
10                       Energy production and conversion
11                       Energy production and conversion
12                       Energy production and conversion
13      Post-translational modification, protein turno...
14      Post-translational modification, protein turno...
15                                          Transcription
16                                          Transcription
17            

In [63]:
%%file eggnog_utils.py
from io import StringIO

import pandas as pd


s = '''A	RNA processing and modification
B	Chromatin Structure and dynamics
C	Energy production and conversion
D	Cell cycle control and mitosis
E	Amino Acid metabolism and transport
F	Nucleotide metabolism and transport
G	Carbohydrate metabolism and transport
H	Coenzyme metabolism
I	Lipid metabolism
J	Translation
K	Transcription
L	Replication and repair
M	Cell wall/membrane/envelope biogenesis
N	Cell motility
O	Post-translational modification, protein turnover, chaperone functions
P	Inorganic ion transport and metabolism
Q	Secondary Structure
T	Signal Transduction
U	Intracellular trafficking and secretion
V	Defense mechanisms
Y	Nuclear structure
W	Extracellular structures
Z	Cytoskeleton
R	General Functional Prediction only
S	Function Unknown'''

cog_categories = pd.read_table(StringIO(s), names=['letter', 'function'], index_col=0, squeeze=True)
cog_categories.head()


COLUMN_NAMES = ['query', 'seed_ortholog', 'evalue', 'score', 'predicted_name', 'go_terms', 
                'kegg_ko', 'bigg_reactions',
           'tax_scope', 'eggnog_ogs', 'best_og', 'cog_cat', 'eggnog_hmmm_desc']


def read_eggnog(filename):
    eggnog = pd.read_table(filename, header=None, names=COLUMN_NAMES)
    eggnog['cog_category_full'] = eggnog.cog_cat.map(
        lambda x: '; '.join([cog_categories[i] for i in x.split(', ')]) if pd.notnull(x) else x)
    return eggnog

Writing eggnog_utils.py


In [68]:
import glob
import os

import eggnog_utils

dfs = []

for filename in glob.glob('../data/00_original/tick*.fake_ref_transcriptome.fasta.emapper.annotations.tsv'):
    
    new_filename = filename.replace('00_original', '012_read_eggnog_output')
    ! mkdir $(dirname $new_filename)
    df = eggnog_utils.read_eggnog(filename)
    df.to_csv(new_filename, sep='\t')
    df['tick_id'] = os.path.basename(filename).split('.')[0]
    dfs.append(df)
annotations = pd.concat(dfs)
print(annotations.shape)
annotations.head()

mkdir: ../data/012_read_eggnog_output: File exists
mkdir: ../data/012_read_eggnog_output: File exists
mkdir: ../data/012_read_eggnog_output: File exists
mkdir: ../data/012_read_eggnog_output: File exists
(15067, 15)


,query,seed_ortholog,evalue,score,predicted_name,go_terms,kegg_ko,bigg_reactions,tax_scope,eggnog_ogs,best_og,cog_cat,eggnog_hmmm_desc,cog_category_full,tick_id
0,PB.3.1|1001_0|path0:1-4172(+)|HQ_sampleytBMi1Q...,6945.ISCW018217-PA,1.400000e-66,260.8,SDC,"GO:0000902,GO:0000904,GO:0001667,GO:0003008,GO...",K16336,NaN,artNOG[6],"0386E@artNOG,0A1US@biNOG,0IW5I@euNOG,0VCQE@meN...",NA|NA|NA,T,Cell surface proteoglycan (By similarity),Signal Transduction,tick0
1,PB.8.1|1008_0|path3:1-3937(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,3.500000e-149,535.0,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel,Intracellular trafficking and secretion,tick0
2,PB.9.1|1008_0|path4:1-3923(+)|HQ_sampleytBMi1Q...,6945.ISCW018083-PA,5.900000e-149,534.3,NaN,NaN,NaN,NaN,artNOG[6],"033W7@artNOG,09WWC@biNOG,0V7UI@meNOG,0XQ0P@NOG...",NA|NA|NA,U,TRIC channel,Intracellular trafficking and secretion,tick0
3,PB.11.2|101_0|path4:1-2908(+)|HQ_sampleytBMi1Q...,6945.ISCW018610-PA,3.300000e-38,165.2,NOLC1,"GO:0000278,GO:0000280,GO:0001071,GO:0002119,GO...",K11294,NaN,meNOG[21],"0A3DQ@biNOG,0VF6Y@meNOG,11U5P@NOG,13M9C@opiNOG...",NA|NA|NA,Y,nucleolar and coiled-body phosphoprotein 1,Nuclear structure,tick0
4,PB.14.1|1011_0|path0:183-1775(-)|HQ_sampleytBM...,7227.FBpp0100183,3.100000e-56,224.9,NaN,"GO:0003674,GO:0003824,GO:0003954,GO:0005575,GO...",K03881,NaN,artNOG[6],"03GA4@artNOG,0AYRX@biNOG,0H2PW@dipNOG,0KFRK@eu...",NA|NA|NA,C,"NADH-Ubiquinone/plastoquinone (complex I), var...",Energy production and conversion,tick0


In [71]:
df['query'].is_unique

True

In [72]:
df['go_terms'].str.split(',')

0       [GO:0003674, GO:0003824, GO:0004407, GO:000548...
1                                                     NaN
2                                                     NaN
3       [GO:0003674, GO:0003724, GO:0003824, GO:000400...
4       [GO:0000003, GO:0001727, GO:0001729, GO:000216...
5       [GO:0000003, GO:0000122, GO:0000226, GO:000027...
6       [GO:0003674, GO:0003824, GO:0004742, GO:000557...
7       [GO:0000302, GO:0002682, GO:0003674, GO:000382...
8                                                     NaN
9       [GO:0000302, GO:0002682, GO:0003674, GO:000382...
10                                                    NaN
11                                                    NaN
12                                                    NaN
13      [GO:0003674, GO:0003824, GO:0003983, GO:000815...
14                                                    NaN
15      [GO:0003674, GO:0003824, GO:0004029, GO:000557...
16      [GO:0001708, GO:0001763, GO:0002009, GO:000216...
17            